In [4]:
import pandas as pd

df = pd.read_parquet("hf://datasets/evalplus/humanevalplus/data/test-00000-of-00001-5973903632b82d40.parquet")

In [7]:
import pstats
import cProfile
results = pd.DataFrame(columns=['task_id', 'entry_point', 'total_time_microseconds'])

def create_executable_script(row, output_filename):
    with open(output_filename, 'w',encoding='utf-8') as f:
        f.write(row['prompt'] + '\n')
        
        solution = row['canonical_solution'].strip().split('\n')
        flag = 0
        for line in solution:
            if flag ==0:
                f.write('    ' + line + '\n')
                flag=1
            else:
                f.write(line + '\n')
        
        f.write('\n')  
        
        f.write(row['test'] + '\n')
        
        f.write('if __name__ == "__main__":\n')
        f.write(f'    check({row["entry_point"]})\n')

for i in range(len(df)):
    row = df.iloc[i]
    task_id = row['task_id']
    entry_point = row['entry_point']
    create_executable_script(row, 'test_code.py')

    def run_time():
        import test_code
    cProfile.run('run_time()', 'output.prof')
    p = pstats.Stats('output.prof')

    total_time_seconds = p.total_tt
    

    total_time_microseconds = total_time_seconds * 1_000_000
    
    temp_df = pd.DataFrame({
        'task_id': [task_id],
        'entry_point': [entry_point],
        'total_time_microseconds': [total_time_microseconds]
    })

    results = pd.concat([results, temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


In [8]:
results

,task_id,entry_point,total_time_microseconds
0,HumanEval/0,has_close_elements,23.2
1,HumanEval/1,separate_paren_groups,25.9
2,HumanEval/2,truncate_number,26.5
3,HumanEval/3,below_zero,26.7
4,HumanEval/4,mean_absolute_deviation,25.1
...,...,...,...
158,HumanEval/159,eat,24.5
159,HumanEval/160,do_algebra,18.8
160,HumanEval/161,solve,27.8
161,HumanEval/162,string_to_md5,19.7


Total Time: 8.600 µs
